# Ciclo de vida de las Referencias 

In [4]:
// practica de referencias

fn main() {
    let x : String = "hola".to_string();
    let y : &String = &x; // y es una referencia a x
    println!("Valor de x: {}", x); // 5
    println!("Valor de y: {}", y); // 5, pero a través de la referencia

    std::mem::drop(x); // ERROR: no se puede eliminar x porque y es una referencia a x, pero si drop no diera error, y seguiría siendo válido

    println!("Valor de y después de drop: {}", y); // 5, la referencia sigue siendo válida
}

main()

Error: cannot move out of `x` because it is borrowed

In [5]:
fn main() {
    let x = 5;
    let y = &x;
    println!("Valor de x: {}", x); // 5
    println!("Valor de y: {}", y); // 5, pero a través de la referencia

    std::mem::drop(x); // ERROR: no se puede eliminar x porque y es una referencia a x, pero si drop no diera error, y seguiría siendo válido

    println!("Valor de y después de drop: {}", y); // 5, la referencia sigue siendo válida
}
main()

Valor de x: 5
Valor de y: 5
Valor de y después de drop: 5


()

### Por qué `std::mem::drop(x)` no dio error con un primitivo

Los tipos primitivos (`i32`, `f64`, `bool`, etc.) implementan el trait `Copy`, lo que significa que **nunca se mueven, siempre se copian**. Entonces `drop(x)` descarta una copia de `x`, no el original. Por eso `y` sigue siendo válida — nunca hubo riesgo de dangling reference.

Con un `String` sí daría error:

```rust
fn main() {
    let x = String::from("hola");
    let y = &x;
    std::mem::drop(x); // ERROR: no se puede mover x porque y lo está prestando
    println!("{}", y);
}
```


### Ciclo de vida de una referencia

El ciclo de vida (lifetime) de una referencia va desde donde se crea hasta la **última vez que se usa**. El compilador lo calcula solo en la mayoría de los casos:


In [8]:

fn main() {
    let x = String::from("hola");

    let y = &x;           // y nace aquí
    println!("{}", y);    // y muere aquí — última vez que se usa

    // A partir de acá y ya no existe, aunque técnicamente esté en scope
    // por eso esto es válido:
    let z = &x;            // x se puede mover, y ya no la está prestando
    println!("{}", z);
}

main()


hola
hola


()


### Referencia mutable bloquea todo lo demás

Cuando existe una referencia mutable (`&mut`), el compilador **bloquea cualquier otra referencia** al mismo valor, sea mutable o inmutable. Es la regla más estricta del borrow checker:


In [9]:

fn main() {
    let mut x = String::from("hola");

    let r_mut = &mut x;       // referencia mutable

    // let r_inmut = &x;      // ERROR: no puede coexistir con &mut
    // let r_mut2 = &mut x;   // ERROR: tampoco dos &mut a la vez

    r_mut.push_str(" mundo");
    println!("{}", r_mut);

    // Acá r_mut ya murió (última vez que se usó arriba)
    // entonces ahora sí podemos crear nuevas referencias:
    let r_inmut = &x;
    println!("{}", r_inmut); // OK
}

main()


hola mundo
hola mundo


()


### Múltiples referencias inmutables sí conviven

La única combinación permitida es **varias `&` al mismo tiempo**, siempre que no haya ninguna `&mut`:


In [10]:

fn main() {
    let x = String::from("hola");

    let r1 = &x;
    let r2 = &x;
    let r3 = &x;

    println!("{} {} {}", r1, r2, r3); // OK, todas leen, ninguna modifica
}

main()

hola hola hola


()


### Resumen de las reglas del borrow checker

| Situación | ¿Permitido? |
|---|---|
| Varias `&` al mismo tiempo | ✅ |
| Una `&mut` sola | ✅ |
| `&mut` + cualquier otra referencia | ❌ |
| Dos `&mut` al mismo tiempo | ❌ |
| Usar referencia después del `drop` del dueño | ❌ (salvo tipos `Copy`) |

La lógica detrás es simple: **múltiples lecturas simultáneas son seguras, pero una escritura debe ser exclusiva**.